In [ ]:
library(readr)
library(dplyr)

In [ ]:
df2<-read.csv("c:/stats202/Objective2_process_data_allpatientID_studyAtoE.csv")

# KNN

In [ ]:
#Chose the Pmean, Nmean,Gmean and PANSS score as the varaibles
df3 = df2[2:5]

In [ ]:
set.seed(2345)
library(animation)
kmeans.ani(df3, 10)

# Finding optimal K

In [ ]:
# Construct a function to compute the total within clusters sum of squares
kmean_withinss <- function(k) {
    cluster <- kmeans(df3, k)
    return (cluster$tot.withinss)
}

In [ ]:
kmean_withinss(2)

In [ ]:
#Run the algorithm n times  
#Set maximum cluster 
#sapply(2:max_k, kmean_withinss): Run the function kmean_withinss() over a range 2:max_k, i.e. 2 to 20.
#max_k <-20: Set a maximum number of to 20

max_k <-20
# Run algorithm over a range of k 
wss <- sapply(2:max_k, kmean_withinss)

In [ ]:
#Create a data frame with the results of the algorithm
elbow <-data.frame(2:max_k, wss)

In [ ]:
library(ggplot2)
# Plot the graph with gglop for WSS v.s K
ggplot(elbow, aes(x = 2:max_k, y = wss)) +
    geom_point() +
    geom_line() +
    scale_x_continuous(breaks = seq(1, 30, by = 1))

In [ ]:
pc_cluster_2 <-kmeans(df3, 4)

In [ ]:
#pc_cluster_2$cluster
pc_cluster_2$centers

In [ ]:
pc_cluster_2$size

In [ ]:
# creating the plots of Gmean and PANSS_total
plot_one = ggplot(df3, aes(x=Gmean, y=PANSS_Total , color=as.factor(pc_cluster_2$cluster))) + geom_point()
plot_one 

In [ ]:
o=order(pc_cluster_2$cluster)
cluster = data.frame(Pmean = df3$Pmean,pc_cluster_2$cluster)
write_csv(cluster,"c:/stats202/cluster_studyAtoE_k=4.csv")

In [ ]:
#fviz_cluster(pc_cluster_2, data = df3)

In [ ]:
library(cluster)
clusplot(df3, pc_cluster_2$cluster, main='2D representation of the Cluster solution',
         color=TRUE, shade=TRUE,
         labels=2, lines=0)

In [ ]:
df %>%
  as_tibble() %>%
  mutate(cluster = pc_cluster_2$cluster,
         state = row.names(df3)) %>%
  ggplot(aes(Nmean,PANSS_Total, color = factor(cluster), label = state)) +
  geom_text()

In [ ]:
# Four clusters in 3 dimensions
x = c(rnorm(20, mean = 0), rnorm(20, mean = 3), rnorm(20, mean = 5), rnorm(20, mean = -10))
y = rnorm(80, mean = 0)
z = c(rnorm(40, mean = -5), rnorm(40, mean = 0))
data = cbind(x, y, z)

#png("examples/4_clusters_3d.png")
scatterplot3d(x, y, z)
dev.off()

# Creating heatmap

In [ ]:
center <-pc_cluster_2$centers
center

In [ ]:
library(tidyr)

# create dataset with the cluster number

cluster <- c(1: 4)
center_df <- data.frame(cluster, center)

# Reshape the data

center_reshape <- gather(center_df,features, values, Pmean, Nmean, Gmean, PANSS_Total)
center_reshape

In [ ]:
library(RColorBrewer)
# Create the palette
hm.palette <-colorRampPalette(rev(brewer.pal(10, 'RdYlGn')),space='Lab')

In [ ]:
# Plot the heat map
ggplot(data = center_reshape, aes(x = features, y = cluster, fill = values)) +
    scale_y_continuous(breaks = seq(1, 7, by = 1)) +
    geom_tile() +
    coord_equal() +
    scale_fill_gradientn(colours = hm.palette(90)) +
    theme_classic() 